<a href="https://colab.research.google.com/github/tgandor/urban_oculus/blob/master/colab/Validate_on_JPEG_Quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%time
!git clone https://github.com/tgandor/urban_oculus
# first, this doesn't kill runtime
%run urban_oculus/snippets/get_coco_val2017.ipynb
# this may kill runtime
%run urban_oculus/snippets/get_detectron2.ipynb

In [0]:
quality = 50

In [2]:
%%time
!mogrify -verbose -quality {quality} datasets/coco/val2017/*.jpg 2>/dev/null | tqdm --total 5000 >/dev/null

  0% 0/5000 [00:48<?, ?it/s]
CPU times: user 188 ms, sys: 16.1 ms, total: 204 ms
Wall time: 52.8 s


In [3]:
!du -h datasets/coco/val2017

204M	datasets/coco/val2017


In [5]:
%%time
import detectron2.model_zoo
from detectron2.config import get_cfg
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor
cfg = get_cfg()
cfg.merge_from_file(detectron2.model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
predictor = DefaultPredictor(cfg)
data_loader = build_detection_test_loader(cfg, 'coco_2017_val')
evaluator = COCOEvaluator('coco_2017_val', cfg, False)
results = inference_on_dataset(predictor.model, data_loader, evaluator)
results

Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=15.03s).
Accumulating evaluation results...
DONE (t=2.06s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.338
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.500
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.373
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.162
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.365
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.470
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.279
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.397
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.402
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

In [0]:
result_file = f'coco_val2017_rcnn_3x_q_{quality}.json'
import json
with open(result_file, 'w') as f:
  f.write(json.dumps(results['bbox']))
from google.colab import files
files.download(result_file)